In [188]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import anvil.media

In [175]:
import anvil.server

# https://anvil.works/build#app:DITALTQUCEBIFSZL
anvil.server.connect("CWGFBQJJPL5HKTHBTADCUZRE-DITALTQUCEBIFSZL")


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [53]:
# list_ds = tf.data.Dataset.list_files("cards/*.jpg", shuffle=False)
# files = [f.numpy().decode() for f in list_ds]
# for f in files:
#     labels = labels.append({"file": f}, ignore_index=True)
    

In [210]:
labels = pd.read_csv("cards/labels.csv")
def save():
    labels.to_csv("cards/labels.csv",index=False)
    
to_label = {
    "number": [r for i,r in labels.iterrows() if np.isnan(r['number'])],
    "fill": [r for i,r in labels.iterrows() if pd.isnull(r['fill'])],
    "colour": [r for i,r in labels.iterrows() if pd.isnull(r['colour'])],
    "shape": [r for i,r in labels.iterrows() if pd.isnull(r['shape'])],
}

@anvil.server.callable
def remaining():
    return len(to_label['number']) + len(to_label['fill']) + len(to_label['colour']) + len(to_label['shape'])

remaining()

0

In [197]:
@anvil.server.callable
def get_unlabelled_card(type):
    if to_label[type]:
        c = random.choice(to_label[type])
        to_label[type] = [x for x in to_label[type] if x is not c]

        return (c['file'], anvil.media.from_file(c['file'], "image/jpeg"), remaining())
    else:
        return None, None, 0

@anvil.server.callable
def label_card(file, type, label):
    for i,r in labels.iterrows():
        if r['file'] == file:
            labels.loc[i,type] = label
            save()
            break
            

In [212]:

get_unlabelled_card("shape")
#label_card("cards\\card.763.jpg", "colour", np.NaN)

(None, None, 0)